# RAG and Semantic Retrieval on a Single Document

Deep Search allows users to interact with the documents using conversational AI, i.e. you interact with a virtual assistant which answer your questions using the information in the document.

In this example we demonstrate how achive the same interaction programmatically.

### Access required

The content of this notebook requires access to Deep Search capabilities which are not
available on the public access system.

[Contact us](https://ds4sd.github.io) if you are interested in exploring
these Deep Search capabilities.


### GenAI Integration required

When interacting with the virtual assistant, Deep Search requires a connection to a Generative AI API. Currently, we support connections to [watsonx.ai](https://www.ibm.com/products/watsonx-ai) or the IBM-internal GenAI platform BAM.

Deep Search allows custom GenAI configurations for each project.
In the following example you will require to work in a project which has such GenAI capabilities activated.

### Set notebook parameters


In [1]:
from dsnotebooks.settings import DocQANotebookSettings

# notebooks settings auto-loaded from .env / env vars
notebook_settings = DocQANotebookSettings()

PROFILE_NAME = notebook_settings.profile     # the profile to use
PROJ_KEY = notebook_settings.proj_key        # the project to use

# index and doc for doc QA from semantically indexed collection
SEM_ON_IDX_KEY = notebook_settings.sem_on_idx_key
SEM_ON_IDX_DOC_HASH = notebook_settings.sem_on_idx_doc_hash

# index and doc for doc QA from not semantically indexed collection
SEM_OFF_IDX_KEY = notebook_settings.sem_off_idx_key
SEM_OFF_IDX_DOC_HASH = notebook_settings.sem_off_idx_doc_hash

RETR_K = notebook_settings.retr_k            # the number of search results to retrieve
TEXT_WEIGHT = notebook_settings.text_weight  # the weight of lexical search (0.0: semantic-only, 1.0: lexical-only, anything in between: hybrid search)
RERANK = notebook_settings.rerank            # whether to rerank the search results

### Import example dependencies

In [2]:
# Import standard dependenices
import rich

# IPython utilities
from IPython.display import display, Markdown

# Import the deepsearch-toolkit
from deepsearch.cps.client.api import CpsApi
from deepsearch.cps.client.components.elastic import ElasticProjectDataCollectionSource
from deepsearch.cps.queries import DocumentRAGQuery, DocumentSemanticQuery
from deepsearch.cps.queries.results import RAGResult, SearchResult, SearchResultItem


### Connect to Deep Search

In [3]:
api = CpsApi.from_env(profile_name=PROFILE_NAME)


### Utils

In [4]:
def render_provenance_url(
        api: CpsApi,
        coords: ElasticProjectDataCollectionSource,
        retr_item: SearchResultItem,
):
    ## compute URL to the document in the Deep Search UI
    item_index = int(retr_item.path_in_doc[retr_item.path_in_doc.rfind(".")+1:])
    doc_url = api.documents.generate_url(
        document_hash=retr_item.doc_hash,
        data_source=coords,
        item_index=item_index,
    )
    display(Markdown(f"The provenance of the answer can be inspected on the [source document]({doc_url})."))

---

## QA on document within semantically indexed collection

### RAG

If the document is part of a semantically indexed collection (see [Document Collection QA](https://github.com/DS4SD/deepsearch-examples/tree/main/examples/qa_doc_collection) for details),
we can directly do RAG on it as shown below:

In [5]:
question = "Where was the first European IBM lab located?"

# submit natural-language query on document
question_query = DocumentRAGQuery(
    question=question,
    project=PROJ_KEY,
    index_key=SEM_ON_IDX_KEY,
    document_hash=SEM_ON_IDX_DOC_HASH,
)
api_output = api.queries.run(question_query)
rag_result = RAGResult.from_api_output(api_output)

rich.print(rag_result)


RAGResult(
    answers=[
        RAGAnswerItem(
            answer='Adliswil, Switzerland',
            grounding=RAGGroundingInfo(
                items=[
                    SearchResultItem(
                        doc_hash='b30bc667a324ae111d025526563b674a8d3fd869bc07c8fd204aa95b05d41f0c',
                        path_in_doc='main-text.71',
                        passage='In 1956, IBM opened their first European research laboratory in Adliswil, 
Switzerland, near Zurich. The lab moved to its own campus in neighboring Rüschlikon in 1962. The Zurich lab is 
staffed by a multicultural and interdisciplinary team of a few hundred permanent research staff members, graduate 
students and post-doctoral fellows, representing about 45 nationalities. Collocated with the lab is a Client Center
(formerly the Industry Solutions Lab), an executive briefing facility demonstrating technology prototypes and 
solutions.',
                        source_is_text=True
                    )
                ]
            )
        )
    ],
    search_result_items=[
        SearchResultItem(
            doc_hash='b30bc667a324ae111d025526563b674a8d3fd869bc07c8fd204aa95b05d41f0c',
            path_in_doc='main-text.71',
            passage='In 1956, IBM opened their first European research laboratory in Adliswil, Switzerland, near 
Zurich. The lab moved to its own campus in neighboring Rüschlikon in 1962. The Zurich lab is staffed by a 
multicultural and interdisciplinary team of a few hundred permanent research staff members, graduate students and 
post-doctoral fellows, representing about 45 nationalities. Collocated with the lab is a Client Center (formerly 
the Industry Solutions Lab), an executive briefing facility demonstrating technology prototypes and solutions.',
            source_is_text=True
        ),
        SearchResultItem(
            doc_hash='b30bc667a324ae111d025526563b674a8d3fd869bc07c8fd204aa95b05d41f0c',
            path_in_doc='main-text.70',
            passage='IBM Research-Zurich (previously called IBM Zurich Research Laboratory, ZRL) is the European 
branch of IBM Research. It was opened in 1956 and is located in Rüschlikon, near Zurich, Switzerland.',
            source_is_text=True
        ),
        SearchResultItem(
            doc_hash='b30bc667a324ae111d025526563b674a8d3fd869bc07c8fd204aa95b05d41f0c',
            path_in_doc='main-text.8',
            passage="The roots of today's IBM Research began with the 1945 opening of the Watson Scientific 
Computing Laboratory at Columbia University. $^{[4]}$ This was the first IBM laboratory devoted to pure science and
later expanded into additional IBM Research locations in Westchester County, New York, starting in the 1950s, 
$^{[5][6]}$ including the Thomas J. Watson Research Center in 1961. [5][6]",
            source_is_text=True
        ),
        SearchResultItem(
            doc_hash='b30bc667a324ae111d025526563b674a8d3fd869bc07c8fd204aa95b05d41f0c',
            path_in_doc='main-text.22',
            passage='Cambridge-IBM Research and MIT-IBM Watson AI Lab (Cambridge, US)',
            source_is_text=True
        ),
        SearchResultItem(
            doc_hash='b30bc667a324ae111d025526563b674a8d3fd869bc07c8fd204aa95b05d41f0c',
            path_in_doc='main-text.62',
            passage="IBM Research-Haifa, previously known as the Haifa Research Lab (HRL) was founded as a small 
scientific center in 1972. $^{[55]}$ Since then, it has grown into a major lab that leads the development of 
innovative technologies and solutions for the IBM corporation. The lab's offices are situated in three locations 
across Israel: Haifa, Tel Aviv, and Beer Sheva.",
            source_is_text=True
        ),
        SearchResultItem(
            doc_hash='b30bc667a324ae111d025526563b674a8d3fd869bc07c8fd204aa95b05d41f0c',
            path_in_doc='main-text.79',
            passage="1. 'Labs and locations ' (https://research.ibm.com/labs/). IBM Research. Archived 
(https://web.archiv

Additionally, we can generate a provenance URL to the document in the Deep Search UI:

In [6]:
coords = ElasticProjectDataCollectionSource(
    proj_key=PROJ_KEY,
    index_key=SEM_ON_IDX_KEY,
)
render_provenance_url(api=api, coords=coords, retr_item=rag_result.answers[0].grounding.items[0])

The provenance of the answer can be inspected on the [source document](https://sds.app.accelerate.science/projects/b09ae7561a01dc7c4b0fd21a43bfd93d140766d1/library/private/6b70072911ad2794a3844dd44d1705a5ba37ca0b?search=JTdCJTIycHJpdmF0ZUNvbGxlY3Rpb24lMjIlM0ElMjI2YjcwMDcyOTExYWQyNzk0YTM4NDRkZDQ0ZDE3MDVhNWJhMzdjYTBiJTIyJTJDJTIydHlwZSUyMiUzQSUyMkRvY3VtZW50JTIyJTJDJTIyZXhwcmVzc2lvbiUyMiUzQSUyMmZpbGUtaW5mby5kb2N1bWVudC1oYXNoJTNBJTIwJTVDJTIyYjMwYmM2NjdhMzI0YWUxMTFkMDI1NTI2NTYzYjY3NGE4ZDNmZDg2OWJjMDdjOGZkMjA0YWE5NWIwNWQ0MWYwYyU1QyUyMiUyMiUyQyUyMmZpbHRlcnMlMjIlM0ElNUIlNUQlMkMlMjJzZWxlY3QlMjIlM0ElNUIlMjJfbmFtZSUyMiUyQyUyMmRlc2NyaXB0aW9uLmNvbGxlY3Rpb24lMjIlMkMlMjJwcm92JTIyJTJDJTIyZGVzY3JpcHRpb24udGl0bGUlMjIlMkMlMjJkZXNjcmlwdGlvbi5wdWJsaWNhdGlvbl9kYXRlJTIyJTJDJTIyZGVzY3JpcHRpb24udXJsX3JlZnMlMjIlNUQlMkMlMjJpdGVtSW5kZXglMjIlM0EwJTJDJTIycGFnZVNpemUlMjIlM0ExMCUyQyUyMnNlYXJjaEFmdGVySGlzdG9yeSUyMiUzQSU1QiU1RCUyQyUyMnZpZXdUeXBlJTIyJTNBJTIyc25pcHBldHMlMjIlMkMlMjJyZWNvcmRTZWxlY3Rpb24lMjIlM0ElN0IlMjJyZWNvcmQlMjIlM0ElN0IlMjJpZCUyMiUzQSUyMmIzMGJjNjY3YTMyNGFlMTExZDAyNTUyNjU2M2I2NzRhOGQzZmQ4NjliYzA3YzhmZDIwNGFhOTViMDVkNDFmMGMlMjIlN0QlMkMlMjJpdGVtSW5kZXglMjIlM0E3MSU3RCU3RA%3D%3D).

Let us try out a different question on our document.
Here we also illustrate some further parameters the user can optionally set:
- `retr_k`: number of items to retrieve
- `text_weight`: weight of lexical search (`0.0`: fully semantic search, `1.0`: fully lexical search, anything in-between: hybrid search)
- `rerank`: whether to rerank the retrieval results

In [7]:
question = "How many research labs does IBM have?"

# submit natural-language query on document
question_query = DocumentRAGQuery(
    question=question,
    project=PROJ_KEY,
    index_key=SEM_ON_IDX_KEY,
    document_hash=SEM_ON_IDX_DOC_HASH,

    # optional params:
    retr_k=RETR_K,
    # text_weight=TEXT_WEIGHT,
    # rerank=RERANK,
)
api_output = api.queries.run(question_query)
rag_result = RAGResult.from_api_output(api_output)

rich.print(rag_result)

RAGResult(
    answers=[
        RAGAnswerItem(
            answer='IBM has 12 research labs.',
            grounding=RAGGroundingInfo(
                items=[
                    SearchResultItem(
                        doc_hash='b30bc667a324ae111d025526563b674a8d3fd869bc07c8fd204aa95b05d41f0c',
                        path_in_doc='main-text.17',
                        passage='IBM currently has 19 research facilities spread across 12 laboratories on six 
continents: [32]',
                        source_is_text=True
                    )
                ]
            )
        )
    ],
    search_result_items=[
        SearchResultItem(
            doc_hash='b30bc667a324ae111d025526563b674a8d3fd869bc07c8fd204aa95b05d41f0c',
            path_in_doc='main-text.17',
            passage='IBM currently has 19 research facilities spread across 12 laboratories on six continents: 
[32]',
            source_is_text=True
        ),
        SearchResultItem(
            doc_hash='b30bc667a324ae111d025526563b674a8d3fd869bc07c8fd204aa95b05d41f0c',
            path_in_doc='main-text.79',
            passage="1. 'Labs and locations ' (https://research.ibm.com/labs/). IBM Research. Archived 
(https://web.archiv e.org/web/20221221032216/https://research.ibm.com/labs/) from the original on 21 December",
            source_is_text=True
        ),
        SearchResultItem(
            doc_hash='b30bc667a324ae111d025526563b674a8d3fd869bc07c8fd204aa95b05d41f0c',
            path_in_doc='main-text.3',
            passage='IBM Research is the research and development division for IBM, an American multinational 
information technology company headquartered in Armonk, New York, with operations in over 170 countries. IBM 
Research is the largest industrial research organization in the world and has twelve labs on six continents. [1]',
            source_is_text=True
        )
    ]
)

In [8]:
render_provenance_url(api=api, coords=coords, retr_item=rag_result.answers[0].grounding.items[0])

The provenance of the answer can be inspected on the [source document](https://sds.app.accelerate.science/projects/b09ae7561a01dc7c4b0fd21a43bfd93d140766d1/library/private/6b70072911ad2794a3844dd44d1705a5ba37ca0b?search=JTdCJTIycHJpdmF0ZUNvbGxlY3Rpb24lMjIlM0ElMjI2YjcwMDcyOTExYWQyNzk0YTM4NDRkZDQ0ZDE3MDVhNWJhMzdjYTBiJTIyJTJDJTIydHlwZSUyMiUzQSUyMkRvY3VtZW50JTIyJTJDJTIyZXhwcmVzc2lvbiUyMiUzQSUyMmZpbGUtaW5mby5kb2N1bWVudC1oYXNoJTNBJTIwJTVDJTIyYjMwYmM2NjdhMzI0YWUxMTFkMDI1NTI2NTYzYjY3NGE4ZDNmZDg2OWJjMDdjOGZkMjA0YWE5NWIwNWQ0MWYwYyU1QyUyMiUyMiUyQyUyMmZpbHRlcnMlMjIlM0ElNUIlNUQlMkMlMjJzZWxlY3QlMjIlM0ElNUIlMjJfbmFtZSUyMiUyQyUyMmRlc2NyaXB0aW9uLmNvbGxlY3Rpb24lMjIlMkMlMjJwcm92JTIyJTJDJTIyZGVzY3JpcHRpb24udGl0bGUlMjIlMkMlMjJkZXNjcmlwdGlvbi5wdWJsaWNhdGlvbl9kYXRlJTIyJTJDJTIyZGVzY3JpcHRpb24udXJsX3JlZnMlMjIlNUQlMkMlMjJpdGVtSW5kZXglMjIlM0EwJTJDJTIycGFnZVNpemUlMjIlM0ExMCUyQyUyMnNlYXJjaEFmdGVySGlzdG9yeSUyMiUzQSU1QiU1RCUyQyUyMnZpZXdUeXBlJTIyJTNBJTIyc25pcHBldHMlMjIlMkMlMjJyZWNvcmRTZWxlY3Rpb24lMjIlM0ElN0IlMjJyZWNvcmQlMjIlM0ElN0IlMjJpZCUyMiUzQSUyMmIzMGJjNjY3YTMyNGFlMTExZDAyNTUyNjU2M2I2NzRhOGQzZmQ4NjliYzA3YzhmZDIwNGFhOTViMDVkNDFmMGMlMjIlN0QlMkMlMjJpdGVtSW5kZXglMjIlM0ExNyU3RCU3RA%3D%3D).

### Semantic retrieval

Besides RAG, which includes natural language generation, a user may only be interested in
the semantic retrieval part.

This can be obtained very similarly to RAG, as shown below:

In [9]:
question = "Where was the first European IBM lab located?"

# submit natural-language query on document
question_query = DocumentSemanticQuery(
    question=question,
    project=PROJ_KEY,
    index_key=SEM_ON_IDX_KEY,
    document_hash=SEM_ON_IDX_DOC_HASH,

    # optional params:
    retr_k=RETR_K,
    # text_weight=TEXT_WEIGHT,
    # rerank=RERANK,
)
api_output = api.queries.run(question_query)
rag_result = SearchResult.from_api_output(api_output)

rich.print(rag_result)

SearchResult(
    search_result_items=[
        SearchResultItem(
            doc_hash='b30bc667a324ae111d025526563b674a8d3fd869bc07c8fd204aa95b05d41f0c',
            path_in_doc='main-text.71',
            passage='In 1956, IBM opened their first European research laboratory in Adliswil, Switzerland, near 
Zurich. The lab moved to its own campus in neighboring Rüschlikon in 1962. The Zurich lab is staffed by a 
multicultural and interdisciplinary team of a few hundred permanent research staff members, graduate students and 
post-doctoral fellows, representing about 45 nationalities. Collocated with the lab is a Client Center (formerly 
the Industry Solutions Lab), an executive briefing facility demonstrating technology prototypes and solutions.',
            source_is_text=True
        ),
        SearchResultItem(
            doc_hash='b30bc667a324ae111d025526563b674a8d3fd869bc07c8fd204aa95b05d41f0c',
            path_in_doc='main-text.8',
            passage="The roots of today's IBM Research began with the 1945 opening of the Watson Scientific 
Computing Laboratory at Columbia University. $^{[4]}$ This was the first IBM laboratory devoted to pure science and
later expanded into additional IBM Research locations in Westchester County, New York, starting in the 1950s, 
$^{[5][6]}$ including the Thomas J. Watson Research Center in 1961. [5][6]",
            source_is_text=True
        ),
        SearchResultItem(
            doc_hash='b30bc667a324ae111d025526563b674a8d3fd869bc07c8fd204aa95b05d41f0c',
            path_in_doc='main-text.70',
            passage='IBM Research-Zurich (previously called IBM Zurich Research Laboratory, ZRL) is the European 
branch of IBM Research. It was opened in 1956 and is located in Rüschlikon, near Zurich, Switzerland.',
            source_is_text=True
        )
    ]
)

## RAG on document not in semantically indexed collection

### Ingestion

In the cell below we show how to semantically index a single document:

In [10]:
from deepsearch.cps.client.components.documents import SemIngestPrivateDataDocumentSource

# prepare collection coordinates
coords = ElasticProjectDataCollectionSource(
    proj_key=PROJ_KEY,
    index_key=SEM_OFF_IDX_KEY,
)

# launch the ingestion of the document for DocumentQA
task = api.documents.semantic_ingest(
    project=PROJ_KEY,
    data_source=SemIngestPrivateDataDocumentSource(
        source=coords,
        document_hash=SEM_OFF_IDX_DOC_HASH,

        # optional params:
        retr_k=RETR_K,
        # text_weight=TEXT_WEIGHT,
        # rerank=RERANK,
    ),
)

# wait for the ingestion task to finish
api.tasks.wait_for(task.proj_key, task.task_id)

{'ing_out': {}}

Once the document has been semantically ingested, we can run both RAG and semantic retrieval queries against it, as shown below.

### RAG

In [11]:
question = "Which company created the first game console?"

# submit natural-language query on document
question_query = DocumentRAGQuery(
    question=question,
    project=PROJ_KEY,
    document_hash=SEM_OFF_IDX_DOC_HASH,

    # optional params:
    retr_k=RETR_K,
    # text_weight=TEXT_WEIGHT,
    rerank=RERANK,
)
api_output = api.queries.run(question_query)
rag_result = RAGResult.from_api_output(api_output)

rich.print(rag_result)


RAGResult(
    answers=[
        RAGAnswerItem(
            answer='Magnavox',
            grounding=RAGGroundingInfo(
                items=[
                    SearchResultItem(
                        doc_hash='029210df929c78e70d74e6f141a46d8326905ce58562f2081819c80c3921d5a3',
                        path_in_doc='main-text.1',
                        passage="Magnavox (Latin for 'great voice ', stylized as MAGNAVOX) is an American 
electronics company that since 1975 has been a subsidiary of the Dutch electronics corporation Philips. [1] The 
predecessor to Magnavox was founded in 1911 by Edwin Pridham and Peter L. Jensen, coinventors of the moving-coil 
loudspeaker at their lab in Napa, California, under United States Patent number 1,105,924 for telephone receivers. 
$^{[2]}$ Six decades later, Magnavox produced the Odyssey, the world's first home video game console.",
                        source_is_text=True
                    )
                ]
            )
        )
    ],
    search_result_items=[
        SearchResultItem(
            doc_hash='029210df929c78e70d74e6f141a46d8326905ce58562f2081819c80c3921d5a3',
            path_in_doc='main-text.1',
            passage="Magnavox (Latin for 'great voice ', stylized as MAGNAVOX) is an American electronics company 
that since 1975 has been a subsidiary of the Dutch electronics corporation Philips. [1] The predecessor to Magnavox
was founded in 1911 by Edwin Pridham and Peter L. Jensen, coinventors of the moving-coil loudspeaker at their lab 
in Napa, California, under United States Patent number 1,105,924 for telephone receivers. $^{[2]}$ Six decades 
later, Magnavox produced the Odyssey, the world's first home video game console.",
            source_is_text=True
        ),
        SearchResultItem(
            doc_hash='029210df929c78e70d74e6f141a46d8326905ce58562f2081819c80c3921d5a3',
            path_in_doc='main-text.9',
            passage='In 1972 Magnavox introduced the Odyssey, the first video game console. $^{[10]}$ In 1975, the 
Magnavox Company was acquired by Philips of the Netherlands to ensure nationwide distribution for their VLP (later 
renamed LaserVision) Videodisc technology, and all Philips consumer electronics in the US under the Norelco name 
began rebranding them under the Magnavox name; $^{[11]}$ Philips acquired the similar-sounding company Philco in 
1981, and Philips was able to freely use the Philips name, alternating with the Magnavox name for some electronics,
with the personal care business continuing to use the Norelco name.',
            source_is_text=True
        ),
        SearchResultItem(
            doc_hash='029210df929c78e70d74e6f141a46d8326905ce58562f2081819c80c3921d5a3',
            path_in_doc='main-text.6',
            passage='Industry, Subsidiary = Consumer electronics. Founded, Subsidiary = 1917 Napa, California, 
U.S.. Founder, Subsidiary = Edwin Pridham Peter L. Jensen. Headquarters, Subsidiary = Knoxville, Tennessee, U.S.. 
Products, Subsidiary = Odyssey and its successors TVs Speakers Blu-ray and DVD. Parent, Subsidiary = Batteries 
Philips',
            source_is_text=False
        )
    ]
)

In [12]:
render_provenance_url(api=api, coords=coords, retr_item=rag_result.answers[0].grounding.items[0])

The provenance of the answer can be inspected on the [source document](https://sds.app.accelerate.science/projects/b09ae7561a01dc7c4b0fd21a43bfd93d140766d1/library/private/b4edbe66a8b8fe2ebed7e20d4d7b9335c48b45b0?search=JTdCJTIycHJpdmF0ZUNvbGxlY3Rpb24lMjIlM0ElMjJiNGVkYmU2NmE4YjhmZTJlYmVkN2UyMGQ0ZDdiOTMzNWM0OGI0NWIwJTIyJTJDJTIydHlwZSUyMiUzQSUyMkRvY3VtZW50JTIyJTJDJTIyZXhwcmVzc2lvbiUyMiUzQSUyMmZpbGUtaW5mby5kb2N1bWVudC1oYXNoJTNBJTIwJTVDJTIyMDI5MjEwZGY5MjljNzhlNzBkNzRlNmYxNDFhNDZkODMyNjkwNWNlNTg1NjJmMjA4MTgxOWM4MGMzOTIxZDVhMyU1QyUyMiUyMiUyQyUyMmZpbHRlcnMlMjIlM0ElNUIlNUQlMkMlMjJzZWxlY3QlMjIlM0ElNUIlMjJfbmFtZSUyMiUyQyUyMmRlc2NyaXB0aW9uLmNvbGxlY3Rpb24lMjIlMkMlMjJwcm92JTIyJTJDJTIyZGVzY3JpcHRpb24udGl0bGUlMjIlMkMlMjJkZXNjcmlwdGlvbi5wdWJsaWNhdGlvbl9kYXRlJTIyJTJDJTIyZGVzY3JpcHRpb24udXJsX3JlZnMlMjIlNUQlMkMlMjJpdGVtSW5kZXglMjIlM0EwJTJDJTIycGFnZVNpemUlMjIlM0ExMCUyQyUyMnNlYXJjaEFmdGVySGlzdG9yeSUyMiUzQSU1QiU1RCUyQyUyMnZpZXdUeXBlJTIyJTNBJTIyc25pcHBldHMlMjIlMkMlMjJyZWNvcmRTZWxlY3Rpb24lMjIlM0ElN0IlMjJyZWNvcmQlMjIlM0ElN0IlMjJpZCUyMiUzQSUyMjAyOTIxMGRmOTI5Yzc4ZTcwZDc0ZTZmMTQxYTQ2ZDgzMjY5MDVjZTU4NTYyZjIwODE4MTljODBjMzkyMWQ1YTMlMjIlN0QlMkMlMjJpdGVtSW5kZXglMjIlM0ExJTdEJTdE).

### Semantic retrieval

In [13]:
question = "Which company created the first game console?"

# submit natural-language query on document
question_query = DocumentSemanticQuery(
    question=question,
    project=PROJ_KEY,
    document_hash=SEM_OFF_IDX_DOC_HASH,

    # optional params:
    retr_k=RETR_K,
    # text_weight=TEXT_WEIGHT,
    rerank=RERANK,
)
api_output = api.queries.run(question_query)
rag_result = SearchResult.from_api_output(api_output)

rich.print(rag_result)

SearchResult(
    search_result_items=[
        SearchResultItem(
            doc_hash='029210df929c78e70d74e6f141a46d8326905ce58562f2081819c80c3921d5a3',
            path_in_doc='main-text.1',
            passage="Magnavox (Latin for 'great voice ', stylized as MAGNAVOX) is an American electronics company 
that since 1975 has been a subsidiary of the Dutch electronics corporation Philips. [1] The predecessor to Magnavox
was founded in 1911 by Edwin Pridham and Peter L. Jensen, coinventors of the moving-coil loudspeaker at their lab 
in Napa, California, under United States Patent number 1,105,924 for telephone receivers. $^{[2]}$ Six decades 
later, Magnavox produced the Odyssey, the world's first home video game console.",
            source_is_text=True
        ),
        SearchResultItem(
            doc_hash='029210df929c78e70d74e6f141a46d8326905ce58562f2081819c80c3921d5a3',
            path_in_doc='main-text.9',
            passage='In 1972 Magnavox introduced the Odyssey, the first video game console. $^{[10]}$ In 1975, the 
Magnavox Company was acquired by Philips of the Netherlands to ensure nationwide distribution for their VLP (later 
renamed LaserVision) Videodisc technology, and all Philips consumer electronics in the US under the Norelco name 
began rebranding them under the Magnavox name; $^{[11]}$ Philips acquired the similar-sounding company Philco in 
1981, and Philips was able to freely use the Philips name, alternating with the Magnavox name for some electronics,
with the personal care business continuing to use the Norelco name.',
            source_is_text=True
        ),
        SearchResultItem(
            doc_hash='029210df929c78e70d74e6f141a46d8326905ce58562f2081819c80c3921d5a3',
            path_in_doc='main-text.6',
            passage='Industry, Subsidiary = Consumer electronics. Founded, Subsidiary = 1917 Napa, California, 
U.S.. Founder, Subsidiary = Edwin Pridham Peter L. Jensen. Headquarters, Subsidiary = Knoxville, Tennessee, U.S.. 
Products, Subsidiary = Odyssey and its successors TVs Speakers Blu-ray and DVD. Parent, Subsidiary = Batteries 
Philips',
            source_is_text=False
        )
    ]
)